In [1]:
# Import data and packages
import pandas as pd, numpy as np
import json

from os import listdir
from os.path import isfile, join

import scispacy, spacy
nlp = spacy.load('en_core_sci_md')

In [2]:
fileDirectory = '/home/rahul/DeepBlue/Projects/CORD-19/noncomm_use_subset'
jsonFiles = [f for f in listdir(fileDirectory) if isfile(join(fileDirectory,f))]
print(jsonFiles)

['df1b6713549bcbe22ec020aaf62275280f096b9a.json', '817a885e9613363e08ef920a9e5826a4cb9e1c1e.json', '3de90c8c18dbed90408c4a5cc5cb94f856c9f637.json', '14e878a6400d8320e2c781a76edc4a14f4cec1df.json', '8ee4731a0d1dabf9a61a177aed392c62ca6fb69b.json', 'e68ff81b1cbbfd6bb525f60a9efdfacf899dabd5.json', '365b8f727f890a4833a5ab7ca03d443022e7faf2.json', 'b76b1127689c03ddc50a70b8dc0057dac74fb805.json', '4135ace30ea08a669730959e0f98a4af06537d07.json', 'b43fc42df698fbc829757a0dac65d9bce3d8f054.json', '9369b15083026f2929d6eb01a67999d1800e2ec0.json', 'ba581ccb585036d6220cfb461733c94584326d96.json', '1f26b5e8291ea1ddc0a3131e8f93f7272093799e.json', '30d2d2d37c2a884db55ab21d1cbd92abcff95fa2.json', 'db71883643ea4dbc79a8f3e5248811d9c3385c3e.json', '4af614b04d5df393575c749cd953aa3855fa8b44.json', '6fe5c74868af62cfacb629af957915de0b094af9.json', 'ff6d57f2aad99be129432058665b361dc18747e8.json', '5f7bc4da553becc28ba03a2ac4a98133d4689adc.json', 'd5bd631a3627f3ef1fef60860af608871fa52271.json', '7f37181b66f15b8ca2

In [15]:
dictList = []
for f in jsonFiles:
    with open(fileDirectory+'/'+f) as currFile:
        data = json.load(currFile)
    dictList.append(data)
df = pd.DataFrame(dictList)
df.head()

,paper_id,metadata,abstract,body_text,bib_entries,ref_entries,back_matter
0,df1b6713549bcbe22ec020aaf62275280f096b9a,{'title': 'Viability of Pseudomonas aeruginosa...,[{'text': 'Background Person-to-person transmi...,[{'text': 'Communicable respiratory infections...,"{'BIBREF0': {'ref_id': 'b0', 'title': 'Economi...","{'FIGREF0': {'text': 'Log', 'latex': None, 'ty...",[{'text': 'We are grateful to all the particip...
1,817a885e9613363e08ef920a9e5826a4cb9e1c1e,{'title': 'CLINICAL EXPERIMENTAL VACCINE RESEA...,[],"[{'text': 'Virulent NDV, the causative agent o...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Recombi...",{'FIGREF0': {'text': 'Schematic diagram of New...,[]
2,3de90c8c18dbed90408c4a5cc5cb94f856c9f637,"{'title': '', 'authors': [{'first': 'Sangmin',...",[{'text': 'A 3-month-old female Maltese puppy ...,"[{'text': 'In humans, cyclosporiasis and cysto...","{'BIBREF1': {'ref_id': 'b1', 'title': 'Epidemi...",{'FIGREF0': {'text': 'Mean corpuscular volume;...,[{'text': 'All authors declared that there is ...
3,14e878a6400d8320e2c781a76edc4a14f4cec1df,{'title': 'Viral etiology of acute respiratory...,[{'text': 'Background: Acute respiratory infec...,[{'text': 'Introduction. Acute respiratory tra...,"{'BIBREF0': {'ref_id': 'b0', 'title': 'The bur...",{'FIGREF0': {'text': 'Distribution of viruses ...,[]
4,8ee4731a0d1dabf9a61a177aed392c62ca6fb69b,"{'title': '', 'authors': []}",[],"[{'text': '. Since then, several cases of MERS...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Isolati...",{'FIGREF0': {'text': 'An outbreak of Middle Ea...,[{'text': 'We thank to our Critical Care Team ...


In [30]:
titles = []
for i in range(len(df)):
    titles.append(df.metadata[i]['title'])
df2 = pd.DataFrame()
df2['paper_id'] = df['paper_id']
df2['title'] = titles
df2.to_csv('PaperID_Title.csv', index=False)
df2.head()

,paper_id,title
0,df1b6713549bcbe22ec020aaf62275280f096b9a,Viability of Pseudomonas aeruginosa in cough a...
1,817a885e9613363e08ef920a9e5826a4cb9e1c1e,CLINICAL EXPERIMENTAL VACCINE RESEARCH
2,3de90c8c18dbed90408c4a5cc5cb94f856c9f637,
3,14e878a6400d8320e2c781a76edc4a14f4cec1df,Viral etiology of acute respiratory infections...
4,8ee4731a0d1dabf9a61a177aed392c62ca6fb69b,


In [20]:
df.metadata[0]['authors']

[{'first': 'Luke',
  'middle': ['D'],
  'last': 'Knibbs',
  'suffix': '',
  'affiliation': {'laboratory': 'International Laboratory for Air Quality and Health',
   'institution': 'Queensland University of Technology',
   'location': {'settlement': 'Brisbane',
    'region': 'Queensland',
    'country': 'Australia'}},
  'email': ''},
 {'first': 'Graham',
  'middle': ['R'],
  'last': 'Johnson',
  'suffix': '',
  'affiliation': {'laboratory': 'International Laboratory for Air Quality and Health',
   'institution': 'Queensland University of Technology',
   'location': {'settlement': 'Brisbane',
    'region': 'Queensland',
    'country': 'Australia'}},
  'email': ''},
 {'first': 'Timothy',
  'middle': ['J'],
  'last': 'Kidd',
  'suffix': '',
  'affiliation': {'laboratory': '',
   'institution': 'The University of Queensland',
   'location': {'settlement': 'Herston',
    'region': 'Queensland',
    'country': 'Australia'}},
  'email': ''},
 {'first': 'Joyce',
  'middle': [],
  'last': 'Cheney

In [91]:
def RemovePunctuation(text):
#    import string
#    table = str.maketrans("","", string.punctuation)
#    return text.translate(table)
    text = text.replace('-',' ')
    return_text = [c for c in text if (c.isalpha() or c==' ')]
    return_text = ''.join(return_text)
    return return_text

In [92]:
print(RemovePunctuation('aaa.bg!s'))

aaabgs


In [93]:
def RemoveCitationsFromParagraph(df, doc_id, para_id):
    remove_indices = []
    for i in range(len(df['body_text'][doc_id][para_id]['cite_spans'])):
        curr_start = df['body_text'][doc_id][para_id]['cite_spans'][i]['start']
        curr_end = df['body_text'][doc_id][para_id]['cite_spans'][i]['end']
        for k in range(curr_start,curr_end):
            remove_indices.append(k)
    remove_indices = set(remove_indices)
    text_return = ''.join([char for idx,char in enumerate(df['body_text'][doc_id][para_id]['text'])
                           if idx not in remove_indices]) 
    return text_return

In [94]:
def GetPaperAuthors(df, doc_id):
    try:
        authors = df.metadata[doc_id]['authors'][0]['last']
        for i in range(1,len(df.metadata[doc_id]['authors'])):
            authors = authors + ', ' + df.metadata[doc_id]['authors'][i]['last']
    except:
        authors='NA'
    return authors

In [95]:
def ConvertToLemma(text):
    doc = nlp(text)
    return [word.lemma_ for word in doc if not(word.like_num or word.is_stop or word.is_punct or word.is_space)]

In [96]:
doc_id = []
doc_title = []
doc_txt = []
doc_authors = []
for i in range(len(df)):
    doc_id.append(df.paper_id[i])
    doc_title.append(df.metadata[i]['title'])
    doc_authors.append(GetPaperAuthors(df,i))
    temp = []
    for k in range(len(df['body_text'][i])):
        curr_para = RemoveCitationsFromParagraph(df,i,k)
        temp.append(curr_para)
    curr_txt = ' '.join(temp) # This text has no citations in it
    curr_txt = RemovePunctuation(curr_txt) # This text has no punctuation in it
    curr_txt = curr_txt.lower() # This text is all in lower case
    #curr_txt = ConvertToLemma(curr_txt) # This text has word lemmas instead of full words
    doc_txt.append(curr_txt)
df2 = pd.DataFrame()
df2['doc_id'] = doc_id
df2['title'] = doc_title
df2['txt'] = doc_txt
df2['author'] = doc_authors
df2.head()

,doc_id,title,txt,author
0,df1b6713549bcbe22ec020aaf62275280f096b9a,Viability of Pseudomonas aeruginosa in cough a...,communicable respiratory infections are an imp...,"Knibbs, Johnson, Kidd, Cheney, Grimwood, Katte..."
1,817a885e9613363e08ef920a9e5826a4cb9e1c1e,CLINICAL EXPERIMENTAL VACCINE RESEARCH,virulent ndv the causative agent of newcastle ...,NA
2,3de90c8c18dbed90408c4a5cc5cb94f856c9f637,,in humans cyclosporiasis and cystoisosporiasis...,"Lee, Kim, Cheon, Moon, Seo, Jung, Shin, Choi"
3,14e878a6400d8320e2c781a76edc4a14f4cec1df,Viral etiology of acute respiratory infections...,introduction acute respiratory tract infection...,"Masoud, Hanna-Wakim, Zaraket, Araj, Matar, Dbaibo"
4,8ee4731a0d1dabf9a61a177aed392c62ca6fb69b,,since then several cases of mers cov infectio...,NA


In [109]:
print(len('Hello an'))

8


In [111]:
# Create a lemma column where all words are lemmatised
from tqdm.notebook import tqdm

# Increase max length of document NLP will look at
nlp.max_length = 1e7
lemmas = []
for i in tqdm(range(len(df2))):
    doc = nlp(df2.txt[i])
    if(len(doc)>nlp.max_length): # Clip documents to max length
        doc = doc[0:nlp.max_length]
    doclemma = []
    for token in doc:
        if(token.lemma_ not in doclemma):
            doclemma.append(token.lemma_)
    lemmas.append(doclemma)
df2['lemmas'] = lemmas

In [112]:
df2.to_pickle("LemmatisedDF_UniqueTokens.pkl")

In [113]:
# Create a lemma column where all words are lemmatised
from tqdm.notebook import tqdm

# Increase max length of document NLP will look at
nlp.max_length = 1e7
lemmas = []
for i in tqdm(range(len(df2))):
    doc = nlp(df2.txt[i])
    if(len(doc)>nlp.max_length): # Clip documents to max length
        doc = doc[0:nlp.max_length]
    doclemma = []
    for token in doc:
        doclemma.append(token.lemma_)
    lemmas.append(doclemma)
df2['nonunique_lemmas'] = lemmas